In [ ]:
# TK Version
from tkinter import filedialog
import tkinter as tk
from PIL import ImageTk, Image
import os
import threading

class Node:
    def __init__(self, item=None, link=None):
        self.item = item
        self.link = link


class CircleLinkedList:
    def __init__(self):
        self.root = Node()
        self.tail = self.root
        self.current = self.root

    def append(self, item):
        # 추가할 새노드를 만든다.
        newNode = Node(item)
        # 루트가 비어 있으면 루트와 테일을 새노드로 지정한다.
        if self.root.item == None:
            self.root = newNode
            self.tail = newNode
            newNode.link = self.root
        else:
            # 루트가 비어 있지 않으면 테일 뒤에 새노드를 추가하고 테일을 업데이트한다.
            self.tail.link = newNode
            newNode.link = self.root
            self.tail = newNode

    # 리스트 안에 모든 노드를 프린트 한다.
    def print(self):
        curNode = self.root
        print(curNode.item)
        while curNode.link != self.root:
            curNode = curNode.link
            print(curNode.item)

    # 리스트에 존재하는 노드 수를 구한다.
    def listSize(self):
        curNode = self.root
        cnt = 1
        while curNode.link != self.root:
            curNode = curNode.link
            cnt += 1
        return cnt

    # 리스트에서 item을 가지는 노드를 current 노드로 지정한다.
    def setCurrent(self, item):
        curNode = self.root
        for i in range(self.listSize()):
            if curNode.item != item:
                curNode = curNode.link
            else:
                self.current = curNode
                break

    def getCurrent(self):
        curNode = self.root
        self.current = curNode
        return curNode.item

    # current 노드를 다음 노드로 바꾼다.
    def moveNext(self):
        self.current = self.current.link
        print("현재 위치는 ", self.current.item, "입니다.")
        return self.current.item
        # 여기에 return이 없으니까 moveNext 했더니 None이고, 그러니까 안읽히지!!!!! 휴

    # current 다음에 item을 삽입한다.
    def insert(self, item):
        # 삽입할 새 노드를 만든다.
        newNode = Node(item)
        # current 노드의 링크에 새 노드를 지정하고 새노드의 링크에 current 노드가 연결되어 있던 링크를 지정하여 연결한다.
        _tmp = self.current.link
        self.current.link = newNode
        newNode.link = _tmp
        # 만약, item1이 tail 노드 였다면 새노드를 tail 노드로 재지정한다.
        if self.current == self.tail:
            self.tail = newNode

    # item을 가지는 노드를 삭제한다.
    def delete(self, item):
        delYN = False
        # 루트가 삭제 대상 노드인지 확인한다. 루트를 삭제할 경우, 루트 다음 노드를 루트로 갱신하고 tail 노드의 링크를 갱신된 루트로 재지정한다.
        curNode = self.root
        if curNode.item == item:
            self.root = self.root.link
            self.tail.link = self.root
            delYN = True
        # 루트가 삭제 대상이 아닐 경우, 끝까지 탐색하면서 item이 존재하는 노드를 찾는다.
        # 찾았다면 preNode의 링크에 대상노드 링크를 넘겨준다. 만약, 대상 노드가 tail노드라면 tail 노드를 preNode로 변경한다.
        else:
            while curNode.link != self.root:
                preNode = curNode
                curNode = curNode.link
                if curNode.item == item:
                    preNode.link = curNode.link
                    if curNode == self.tail:
                        self.tail = preNode
                    delYN = True
        # 삭제를 하지 못했으면 error message를 출력한다.
        if delYN == False: print("delete failed")

    # def shuffle(self):
    #     import random
    #     curNode = self.root
    #     list_item = []
    #     while curNode.link != self.root:
    #         list_item.append(curNode.item)
    #         curNode = curNode.link
    #     list_item.append(curNode.item)
    #     random.shuffle(list_item)
    #     s = CircleLinkedList()
    #     for item in list_item:
    #         s.append(item)
    #     return s

    # shuffle 하면 새로운 circlelinkedlist 생기는 거 아님? 
    # 이렇게 따로 만들 필요 없음. 그냥 bShuffleClick에서 바로 할 수 있음.




class Window():
    def __init__(self):
        self.window=tk.Tk()
        self.window.title("Image Viewer")
        self.window.geometry("640x480")
        self.window.resizable(False, False)

        # 메뉴바를 윈도우에 추가한다.
        menubar = tk.Menu(self.window)
        menu_1 = tk.Menu(menubar, tearoff=0)   # 이렇게 하면 위에 메뉴창에 추가됨.
        menu_1.bind('<<MenuSelect>>')

        menu_1.add_command(label="Open", command=self.dirSelect)
        menu_1.add_command(label="Close", command=self.close)
        menubar.add_cascade(label="File", menu=menu_1)

        self.window.config(menu=menubar)

        # 이미지 라벨을 추가한다.
        self.imgLabel = tk.Label(self.window, width=400, height=400, relief='solid')
        self.imgLabel.pack()   # pack이 호출되어야만 위젯이 화면에 나타난다.

        #self.window.mainloop()   # mainloop 메소드는 마우스 버튼과 같은 이벤트를 처리하면서 사용자가 윈도우를 닫을 때까지 반복한다.
        #여기에 mainloop작성하면 버튼 추가 입력 안됨.

        #버튼을 추가한다.
        b1 = tk.Button(self.window, text='Next', command=self.bNextClick)
        b2 = tk.Button(self.window, text='Shuffle', command=self.bShuffleClick)
        b3 = tk.Button(self.window, text='SlideShow', command=self.bSlideShowClick)

        # b1.place(self.window, x=200, y=400)   # 왜 오류가 뜨지? 이렇게 하는 게 아닌가,,?
        # b2.place(self.window, x=320, y=400)
        # b3.place(self.window, x=440, y=400)

        b1.pack(side='left', ipadx=100)
        b2.pack(side='left', ipadx=70)
        b3.pack(side='left', ipadx=75)
        # side=LEFT 하면 오류뜸. side='left' 이런 식으로 해야 함.
        # ipadx를 이렇게 설정해야 위젯이 나름 예쁘게 나옴. 제대로 된 곳에 배치하는 법을 모르겠음,,
        self.window.mainloop()

    # 메뉴에서 close 가 선택되었을 때 수행한다.
    def close(self):
        self.window.quit()
        self.window.destroy()

    # 파일을 선택한다.
    def dirSelect(self):
        global a
        self.pngLists = CircleLinkedList()
        selDir = filedialog.askdirectory(initialdir="./", title="Select directory")
        a = selDir
        print('선택경로', selDir)
        self.pngfiles = [x for x in os.listdir(selDir) if x.endswith(".jpg")]
        # os 모듈은 운영체제에서 제공되는 기능들을 수행할 수 있게 해줌.
        # 그 중 listdir 함수는 해당 경로에 들어있는 파일 또는 디렉터리를 리스트로 반환해줌.
        # 여기서는 selDir 파일중에 '.jpg'로 끝나는 모든 파일을 pngfiles로 저장하고 아래 동작을 수행(?)
        # .jpg 파일만 걸러주는 역할(?)
        print('파일명', self.pngfiles)
        for fname in self.pngfiles:
            self.pngLists.append(selDir + "/" + fname)   # 경로를 저장하기 위해 다 '/' 넣는 거,,?!

        selFile = self.pngLists.getCurrent()
        self.fileSelect(selFile)


    def fileSelect(self, selFile):
        # 초기 디렉토리를 루트로 지정하고 파일을 선택하면 해당 파일명을 selFile에 입력한다.
        print(selFile)
        self.image = Image.open(selFile)
        print(self.image.size[0], self.image.size[1])
        # 해당 이미지의 크기를 400, 400으로 resize 한다.
        if self.image.size[1] > self.image.size[0]:
            hSize = int((400 * self.image.size[0] / self.image.size[1]))
            vSize = 400
        else:
            hSize = 400
            vSize = int((400 * self.image.size[1] / self.image.size[0]))

        self.image = self.image.resize((hSize,vSize), Image.ANTIALIAS)
        self.image = ImageTk.PhotoImage(self.image)
        self.imgLabel.config(image=self.image)

        # 단순히 파일 하나만 가  져오는 게 아니라, 파일 여러개를 가져와서 그 파일들을 CircleLinkedList로 연결해야함.
        # 파일들을 저장할 리스트가 필요한 거 아닌가..?

    # next 버튼을 눌렀을 때 수행한다.
    def bNextClick(self):
        selFile = self.pngLists.moveNext()
        self.fileSelect(selFile)

    # shuffle 버튼을 눌렀을 때 수행한다.
    def bShuffleClick(self):
        # new_pngList = self.pngLists.shuffle()   # 왜 이렇게 하면 Nonetype이 되는거야,,?!
        # selFile = new_pngList.moveNext()
        # self.fileSelect(selFile)

        # 이렇게 하면 원래 리스트에서 다 빼서 새로운 circlelinkedlist에 집어넣는 게 돼버림/

        import random
        pngLists = CircleLinkedList()
        random.shuffle(self.pngfiles)
        for fname in self.pngfiles:
            pngLists.append(a + "/" + fname)
            # 여기서 저 selDir을 어떻게 사용해야하지,,? --> selDir이 string형이었음. 그래서 a로 받고 그 a를 전역변수로 설정해준 후 가져옴.
            # 성공!

        selFile = pngLists.getCurrent()
        self.fileSelect(selFile)


    # slideshow 버튼을 눌렀을 때 수행한다.
    def bSlideShowClick(self):
        # # 파일 넘어가는 시간 대기 후 자동으로 self.bNextClick 실행(?)
        # # 일정 시간 대기 구현 어떻게,,? -> after 함수 사용
        # selFile = self.pngLists.current.item
        # for i in range(self.pngLists.listSize()):
        #     self.window.after(1000, self.bNextClick)
        #     self.fileSelect(selFile)
        # # 어떻게 자동으로 넘기는지 모르겠다..

        # 이렇게 하면 한 번에 슈루룩 넘어가버림.

        # 방법1
        selFile = self.pngLists.moveNext()
        self.fileSelect(selFile)
        self.window.after(3000, self.bSlideShowClick)   # bSlideShowClick을 시간초 두고 실행시키면 자동으로 moveNext가 실행됨!

        # # 방법2
        # selFile = self.pngLists.moveNext()
        # self.fileSelect(selFile)
        # threading.Timer(3, self.bSlideShowClick).start()   # threading이 뭔지 잘 모르지만 이 방법도 됨!

        # 성공!!


a = Window()